Развернуть у себя на компьютере/виртуальной машине/хостинге MongoDB и реализовать функцию, которая будет добавлять только новые вакансии/продукты в вашу базу.

* Написать функцию, которая производит поиск и выводит на экран вакансии с заработной платой больше введённой суммы (необходимо анализировать оба поля зарплаты).

* Любая аналитика. Например matching ваканский с разных площадок

# Установим клиент и сервер MongoDB

In [28]:
#!pip install pymongo
!apt install mongodb > log

In [29]:
!service mongodb start

 * Starting database mongodb
   ...done.


In [30]:
!pip install pymongo

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [31]:
from pymongo import MongoClient

In [23]:
#Создадим клиента для работы с БД
client = MongoClient('localhost:27017')

# Наполним базу данных вакансиями с прошлого урока

In [2]:
#Сделаем функцию парсинга отдельно
def PageParser_HH (url, params , headers):
  vacancies_list = []
  #Получение DOM с сайта
  response = requests.get(url=url, params=params, headers=headers)
  #Извлечение искомых полей
  soup = bs(response.text, 'html.parser')
  vacancies = soup.find_all('div', {'class': 'serp-item'})
  #Оформление списка словарей вакансий  
  for vacancy in vacancies:
    vacancy_name = vacancy.find('a', {'class': 'serp-item__title'}).getText()
    vacancy_link = vacancy.find('a', {'class': 'serp-item__title'})['href']
    vacancy_salary = vacancy.find('span', {'data-qa': 'vacancy-serp__vacancy-compensation'})
    if vacancy_salary is not None:
      vacancy_salary = vacancy_salary.getText()

    vacancy_dict = {
        'name': vacancy_name,
        'link': vacancy_link,
        'salary': vacancy_salary,
        'source': 'HeadHunter (https://hh.ru/)',
        }
    vacancies_list.append(vacancy_dict)
  return vacancies_list

In [3]:
# Функция преобразовывающая строку зарплаты в словарь
def Salary_trace(salary):
  if salary != None:
  #Возьмем строку и очистим от лишних символов
   string = salary.replace('\u202f','').replace(' ','')
   #Вычленим валюту
   currency = string
   for c in ['0' , '1' ,'2' ,'3' ,'4' ,'5' ,'6' ,'7' ,'8' ,'9' ,'от' , 'до' , '–']:
      currency = currency.replace(c,'')

  #Вычленим минимальную ЗП
   if string.find('от') > -1:
     From = int(''.join(i for i in string if not ( i.isalpha() or i =='.')))
   elif string.find('–') > -1:
     From = int((string[:string.find("–")]))
   else: 
     From = None

  #Вычленим максимальную ЗП
   if string.find('до') > -1:
     To = int(''.join(i for i in string if not ( i.isalpha() or i =='.')))
   elif string.find('–') > -1:
     To = int((string[string.find("–")+1:string.find(currency)]))
   else: 
     To = None

  else:
   From = None
   To = None
   currency = None

  salary_dict = {'From': From , 'To' : To , 'currency' : currency }

  return salary_dict

In [4]:
from bs4 import BeautifulSoup as bs
import requests

In [5]:
url = 'https://hh.ru/search/vacancy?'
NotEmpty = True
SearchPage = 6
vacancies_list = []

params = {
    'area': 1,
    'text': 'python',
    'items_on_page': 20,
    'experience' : 'noExperience',
    'page' : SearchPage
}

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/104.0.5112.124',
}

In [6]:
# Перебор страниц сайта с наполнением списка вакансий
while NotEmpty:
  vacancies_list_page = []
  vacancies_list_page = PageParser_HH(url, params, headers)
  vacancies_list = vacancies_list + vacancies_list_page
  params["page"] +=1  #Переход на следующую страницу поиска
  if len(vacancies_list_page) == 0: # Проверка на наличие результата
    NotEmpty = False


In [7]:
# Преобразуем зарплаты в словари зарплат
for i in range(len(vacancies_list)):
  vacancies_list[i]['salary'] = Salary_trace(vacancies_list[i]['salary'])
  # из словаря извлечем в ключи
  vacancies_list[i]['salary_From'] = vacancies_list[i]['salary']['From']
  vacancies_list[i]['salary_To'] = vacancies_list[i]['salary']['To']
  vacancies_list[i]['salary_currency'] = vacancies_list[i]['salary']['currency']
  del vacancies_list[i]['salary']


# Внесем полученные данные в базу:

In [24]:
#Создадим ссылку на БД vacancies_DB
db = client.vacancies_DB

In [25]:
#Создадим раздел БД (Коллекуию) vacancies_collection
collection = db.vacancies_collection

In [76]:
#Функция очистки каталога )для теста)
collection.delete_many({})

In [77]:
#Наполним коллекцию базы данных имеющимися данными
for vacancy in vacancies_list:
  collection.insert_one(vacancy)

In [78]:
#Сравним количество записей в словаре и в каталоге базы данных
print(len(vacancies_list))
print(len(list(collection.find())))

42
42


In [79]:
#Создадим функцию добавления только новых вакансий в базу данных
def add_new_vacancy(vacancy , collection):
  if (len(list(collection.find( { 'name' : vacancy['name']}))) > 0):
    print('Вакансия с таким именем уже есть в каталоге:\n', list(collection.find( { 'name' : vacancy['name']})))
  else:
    collection.insert_one(vacancy)

In [81]:
# Сделаем тестовую вакансию, отсутствующую в списке
TestVac = {'name': 'Вакансия мечты!',
           'link': 'https://www.youtube.com/watch?v=dQw4w9WgXcQ', 
           'source': 'Сверху', 
           'salary_From': 10000, 
           'salary_To': 200000, 
           'salary_currency': 'USD'}

In [80]:
#Добавим существующую вакансию
add_new_vacancy(vacancies_list[2] , collection)

Вакансия с таким именем уже есть в каталоге:
 [{'_id': ObjectId('633eca358d97155f9b91007b'), 'name': 'Стажер-программист 1С', 'link': 'https://hh.ru/vacancy/69956353?from=vacancy_search_list&hhtmFrom=vacancy_search_list&query=python', 'source': 'HeadHunter (https://hh.ru/)', 'salary_From': None, 'salary_To': None, 'salary_currency': None}]


In [82]:
#Добавим новую вакансию
add_new_vacancy(TestVac , collection)

In [83]:
# Еще раз новую вакансию (чтобы проверить ошибку)
add_new_vacancy(TestVac , collection)

Вакансия с таким именем уже есть в каталоге:
 [{'_id': ObjectId('633ed1d38d97155f9b9100a4'), 'name': 'Вакансия мечты!', 'link': 'https://www.youtube.com/watch?v=dQw4w9WgXcQ', 'source': 'Сверху', 'salary_From': 10000, 'salary_To': 200000, 'salary_currency': 'USD'}]


# Написать функцию, которая производит поиск и выводит на экран вакансии с заработной платой больше введённой суммы (необходимо анализировать оба поля зарплаты).

In [91]:
search_salary = int(input('Введите искомую зарплату (руб.): '))


Введите искомую зарплату (руб.): 25000


In [116]:
#Вывод вакансий с ЗП выше введенной
reslist = list(collection.find( {'$or' : [
    {'$and' : [{ '$or' : [ {'salary_From': {'$lt': search_salary}} , {'salary_To': {'$gt': search_salary}} ] },
              {'salary_currency' : 'руб.' }]},
    {'$and' : [{ '$or' : [ {'salary_From': {'$lt': search_salary/60}} , {'salary_To': {'$gt': search_salary/60}} ] },
              {'salary_currency' : 'USD' }]}
                                          ]} ))

print( 'Вакансии с ЗП выше ' , search_salary )
for v in reslist:
  print(v)

Вакансии с ЗП выше  25000
{'_id': ObjectId('633eca358d97155f9b91007d'), 'name': 'Ночной специалист службы поддержки с техническими знаниями (Браузер)', 'link': 'https://hh.ru/vacancy/48221815?from=vacancy_search_list&hhtmFrom=vacancy_search_list&query=python', 'source': 'HeadHunter (https://hh.ru/)', 'salary_From': 15000, 'salary_To': None, 'salary_currency': 'руб.'}
{'_id': ObjectId('633eca358d97155f9b910082'), 'name': 'Преподаватель программирования для детей', 'link': 'https://hh.ru/vacancy/69884058?from=vacancy_search_list&hhtmFrom=vacancy_search_list&query=python', 'source': 'HeadHunter (https://hh.ru/)', 'salary_From': None, 'salary_To': 45000, 'salary_currency': 'руб.'}
{'_id': ObjectId('633eca358d97155f9b910088'), 'name': 'Менеджер отдела ценовой политики', 'link': 'https://hh.ru/vacancy/69870192?from=vacancy_search_list&hhtmFrom=vacancy_search_list&query=python', 'source': 'HeadHunter (https://hh.ru/)', 'salary_From': None, 'salary_To': 75500, 'salary_currency': 'руб.'}
{'_id'